# <center>Desafio 1: 
# <center>Análisis exploratorio de un dataset de precios de propiedades


### Grupo 8:
- Pablo Haedo
- Marx Morales
- Guillermo Portero
- Alejandra Fusco


## Importación de librerías:

In [1]:
#Importamos lirerías:
import os
import numpy as np
import pandas as pd
import re
import math
import seaborn as sns
import matplotlib.pyplot as plt

## 1. Carga de datos:
Levantamos el archivo que contiene los datos de publicaciones de propiedades del año 2017 en la plataforma Properati.

In [2]:
#Levantamos la base de properati:
df = pd.read_csv('properatti.csv')

## 2. Análisis del Dataset:
Hacemos una primera visualización de datos, tomando una muestra de 10 regustros al azar.

In [3]:
# Para mostrar todas las columnas:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 1000)
# Muestra de los datos:
df.head(10)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.00,62000.00,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mataderos_lavadero_patio_inmobiliaria-fedele,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PATIO GRANDE, CON LAVADERO, LIVING COMEDOR CON COCINA INTEGRADA, ARTEFACTO DE COCINA, ALACENA, BAÑO COMPLETO, Y DORMITORIO. TODO EN EXCELENTE ESTADO, PARA HABITAR.NO ES APTO CREDITO Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIKufMJMwHtquzhk=/trim/198x0/smart/filters:strip_icc()/inmobiliariafedele.com/photos/CARHUE%202278-IMG_3400.JPG
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,USD,2646675.00,150000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_departamentos_la-plata_balcon_lavadero_toilette_garage_estrenar_antonini-propiedades_dcp,"Venta de departamento en décimo piso al frente, a estrenar. Living comedor con ventanales hacia el balcón con pisos de madera. Cocina completa con doble mesada, muebles bajo mesada. Lavadero separado. Toilette.Dormitorio con placard. Segundo dormitorio a dividir. Calefacción por radiadorBaño completo.Balcón corrido. Posibilidad de cochera semicubierta. U$D 20.000 Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv45KAKirGL3JM8=/trim/198x0/smart/filters:strip_icc()/antoninipropiedades.com/photos/401-1-7%20y%2037%2010A%201.jpg
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.00,72000.00,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_departamentos_mataderos_lavadero_placard_inmobiliaria-fedele,"2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON LAVADERO, BAÑO COMPLETO Y DORMITORIO CON PLACARD. BAJAS EXPENSAS, EXCELENTE UBICACION Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_Ejwfu6QpW4cH0=/trim/198x0/smart/filters:strip_icc()/inmobiliariafedele.com/photos/GUAMINI%201171-IMG_3403.JPG
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.50,95000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_liniers_patio_g-goffredo,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso.Reciclado a nuevo. ""Muy buen estado"" Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SPyleyeQlIypI8w=/trim/198x0/smart/filters:strip_icc()/ggoffredo.com/photos/GG0040-1563480102.jpg
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,64000.0,USD,1129248.00,64000.00,35.0,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_departamentos_centro_cristina-pavone-negocios-inmobiliarios,"DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURAL, AL CONTRAFREN

Visualizamos la estructura del dataset y la distribución de los datos.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

Vemos la cantidad de NaN por columna:

In [5]:
df.isna().sum()

Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

## 3. Manipulación de los datos:
### 3.1. Eliminamos columnas:
3.1.1. Columna Operación: Todos los registros contienen el mismo valor "Sell". No suma valor almacenar éste campo.

In [6]:
#Operación: todos los registros son venta, 
df.drop(columns=['operation'], inplace=True)

3.1.2. Columna Image_thumbnail: Provee la localización de la imagen de la propiedad, que se muestra en la publicación. Las imágenes no se encuentran activas y de todos modos no suman valor.

In [7]:
#La dirección de la imagen, no nos sirve:
df.drop(columns=['image_thumbnail'], inplace=True)

### 3.2. Exploramos cada columna restante
3.2.1 Ambientes:

Revisamos la distribución de los datos del campo Rooms.

In [8]:
#Cantidad de registros en cada valor:
df['rooms'].value_counts()

3.0     12567
2.0     12433
4.0      7937
1.0      7377
5.0      3782
6.0      1403
7.0       770
8.0       409
10.0      226
9.0       184
11.0       72
12.0       65
13.0       33
14.0       27
15.0       26
17.0       16
16.0       11
20.0       11
22.0        8
18.0        6
21.0        5
30.0        4
25.0        4
19.0        4
32.0        3
23.0        2
24.0        1
27.0        1
29.0        1
31.0        1
28.0        1
Name: rooms, dtype: int64

In [10]:
df.loc[:,["rooms","description","title"]]

,rooms,description,title
0,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PATIO GRANDE, CON LAVADERO, LIVING COMEDOR CON COCINA INTEGRADA, ARTEFACTO DE COCINA, ALACENA, BAÑO COMPLETO, Y DORMITORIO. TODO EN EXCELENTE ESTADO, PARA HABITAR.NO ES APTO CREDITO Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,NaN,"Venta de departamento en décimo piso al frente, a estrenar. Living comedor con ventanales hacia el balcón con pisos de madera. Cocina completa con doble mesada, muebles bajo mesada. Lavadero separado. Toilette.Dormitorio con placard. Segundo dormitorio a dividir. Calefacción por radiadorBaño completo.Balcón corrido. Posibilidad de cochera semicubierta. U$D 20.000 Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37
2,NaN,"2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON LAVADERO, BAÑO COMPLETO Y DORMITORIO CON PLACARD. BAJAS EXPENSAS, EXCELENTE UBICACION Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,NaN,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso.Reciclado a nuevo. ""Muy buen estado"" Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",PH 3 amb. cfte. reciclado
4,NaN,"DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURAL, AL CONTRAFRENTE, EDIFICIO DE POCAS UNIDADES Y MUY BIEN CONSERVADO. EN ESTADO ORIGINAL.-POSEE: 1 DORMITORIOS, 1 BAÑO COMPLETO, LIVING COMEDOR Y COCINA SEPARADA.-EN ESTADO ORIGINAL.- Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA MITRE
5,NaN,"Casa en el perímetro del barrio 338, ubicada en el extremo sur este del barrio, sobre calle 3 de caballería casi Nágera. Vista libre a terreno en frente.Consta de dos dormitorios, baño, living cocina comedor integrado de buen tamaño, lavadero con despensero en aluminio.Patio privado con churrasquera, tapiales y puerta de acceso de servicio. Frente con rejas y pequeño patio con verde, porch de acceso.Gas natural en la puerta. Aire acondicionado frío/calor. Baulera para guardado sobre pasillo. Muy buen estado en general de pisos, revestimientos, pintura, etc.Apto crédito. Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).","Casa Barrio 338. Sobre calle 3 de caballería, casi Nágera. Dos habit. Muy buen estado, patio con tapiales."
6,NaN,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENTE - SIN EXPENSAS! TODO EN PLANTA BAJA DOS DORMITORIOS (pisos de pinotea) - LIVING - COCINA COMEDOR - BAÑO - PATIO - GALPÓN Y LAVADERO EN EL PATIOEN MUY BUEN ESTADO - INMEJORABLE UBICACIÓN - A 1 CUADRA DE LA ESTACIÓN DE MUNRO - CERCA DE MAS DE 6 LINEAS DE COLECTIVOS - A 2 CUADRAS DEL CENTRO UNIVERSITARIO. 106 m2 TOTALES: 78 m2 CUBIERTOS Y 28 m2 DESCUBIERTOSAPTO PROFESIONAL Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO, EXCELENTE UBICACION EN MUNRO"
7,NaN,"EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER LUMINOSO ...PISO ALTO COCINA AMERICANA. BAÑO COMPLETO PILETA ,SOLARIUM, LAUNDRY APTO PROFESIONALPROXIMO AV DEL LIBERTADOR MUY BUENA ZONA !!!! Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).","JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CATEGORIA , VENTA"
8,NaN,"EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER LUMINOSO ...PISO ALTO COCINA AMERICANA. BAÑO COMPLETO PILETA ,SOLARIUM, LAUNDRY APTO PROFESIONALPROXIMO AV DEL LIBERTADOR MUY BUENA ZONA !!!! Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).","JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR , CATEGORIA , VENTA"
9,NaN,"MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWHITE 7637 SECTOR E MONOBLOK 67LISTO PARA ESCRITURARPLANTA BAJA CON PATIO2 DORMITORIOSLI

In [11]:
#Verifico que ninguno tenga cero rooms
df[df['rooms']==0.0]

,Unnamed: 0,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title


In [12]:
#Vemos cómo es un campo desripción
df.loc[np.random.randint(0,high=100000),'description']

' Departamento un ambiente contrafrente. Cocina integrada.Gas.Calefacción individual .Carpintería de aluminio.Edificio con dos Jacuzzi.Cocheras disponibles.David Cipolla PropiedadesBelgrano  // La Imprenta  // Barrancas      Matrículas 4052-CUCICBA / 6065-CMCPSI'

In [13]:
#Vemos si las propiedades con 20 ambientes realmente lo son:
df[df['rooms']>20].loc[:,['rooms','description','title','surface_total_in_m2']]

,rooms,description,title,surface_total_in_m2
877,22.0,"Corredor Responsable: Mario Gabriel Gerez - CUCICBA 6477Contacto: Juan Salvador Córdova - MLS ID # 371159-12Se Vende, solos o en bloque piso en Calle peru, pleno centro. El inmueble funciona actualmente como hotel familiar. Y se entrega libre de ocupantes, a menos que se especifique lo contrario. La renta actual es de aproximadamente $ 50.000 por mes. El piso se compone de la siguiente manera. 472 metros totales 346 metros cubiertos 88 semicubiertos 37 descubiertos Preguntar por la opción de comprar el edificio entero. Martillero y Corredor Público Mario G. Gerez MPCSI 6012 / CUCICBA 6477 Av. Del Libertador 618 Vicente Lopez, 1642 Vicente Lopez, Argentina",Venta Hotel Familiar Calle Peru al 100,NaN
14714,25.0,"Corredor Responsable: Hernan Perrone - CMCPDJMGR 170Contacto: Eduardo Kenny - MLS ID # 420411004-59Departamento muy bien ubicado.Cuenta con un dormitorio amplio y un segundo dormitorio más pequeño, living/comedor, baño completo y cocina.",Departamento en Moreno Centro,NaN
27354,25.0,"Corredor Responsable: Edgardo Tamarit - CPCPI 4228Contacto: Laura Kotoucek - MLS ID # 420401043-79SE VENDE HOSTERÍA A REFACCIONAR EN LA FALDA CÓRDOBA!! Queres invertir?? no lo dudes.... muy buena ubicación, ya que esta a una cuadra de la ruta 38 a tres cuadras del Super Max de la entrada de La Falda. Tiene una vista impresionante a las sierras... Tiene una casa atrás de 3 dormitorios, 2 baños, Comedor, cocina, living y asador 5 piezas con baño privado (de las cuales 3 estan alquiladas x mes) 3 Departamentos: 1 Monoambiente 1 con dos habitaciones, baño living y cocina comedor 1 con una habitación living cocina comedor y baño Llamanos y vamos a verla...",SE VENDE HOSTERIA A REFACCIONAR LA FALDA CORDOBA,NaN
28078,25.0,"Corredor Responsable: DANIELA E. CASTRO - CPCPI 5292Contacto: Juan Pablo Moyano Centeno - MLS ID # 420201171-3El predio tiene 10.300 M2 de los cuales 850 M2 están construidos y EN MUY BUEN ESTADO -Posee cocheras con media sombra (3) -Dos garajes -Pileta con solárium -Al frente amplio jardín, CON CANTEROS Y FLORES, GRAN PARQUE, AÑOSA ARBOLEDA , FRUTALES -El parque en su mayoría es plano. El inmueble cuenta con: -6 baños en suite, 2 baños completos y 2 de servicio, TODOS EN PERFECTO ESTADO. -Posee dos living, ambos amplios y con ventanales al parque. -Sala de estar,posee comedor y sala, gran living, estar comedor, y ambientes muy amplios. -Cocina COMPLETA Y PREPARADA PARA COCINA ALTERNATIVA. Posee amplia ante cocina con placares -Dormitorios:Cuenta con 10 habitaciones amplias, 6 de ellas con baño propio. -Sus Galerías:Posee dos hermosas galerías de estilo colonial con vista al parque y a su pileta. -Asador -Lavadero. Posee amplio espacio. A metros de la misma nos encontramos con ...",SE VENDE CASONA APTA HOTEL EN VALLE HERMOSO,NaN
33818,30.0,"Corredor Responsable: Matiaz Ruiz Moreno - CPCPI 4740Contacto: Fabiana Pereyra - MLS ID # 420581024-86VENDO OPORTUNIDAD COSTO-CALIDAD APART-HOTEL SOBRE COSTANERA SANTA ROSA DE CALAMUCHITA Oportunidad de Inversión en Santa Rosa de Calamuchita . Enclavado frente al río y a pocas cuadras del centro de la ciudad. Sobre la costanera de fácil acceso desde ruta 5. Actualmente en actividad . Superficie de 650 m2 en dos plantas sobre terreno de 1300 m2 CARACTERÍSTICAS: Recepción vidriada en el ingreso principal con baño( mobiliario de oficina) 9 departamentos Lavadero ( lavarropas industrial, secadora Marva y plancha manual) Solarium Sala de juego (mesa de pool, metegol y mesa de ping-pong) Depósito (herramienta, máquina de cesped, aspiradora e hidro-lavadora) Quincho ( 3 parrillas con baño, mesas, sillas, tablas, utensilios) Piscina climatizada (sistema solar y cerramiento telescópico y jacuzzi) Cochera y tender (acceso por portón automático) Terraza ( vista al Río para tiempo de d...",APART HOTEL COSTANERA SANTA ROSA DE CALAMUCHITA,NaN
40104,30.0,"Corredor Responsable: Carlos Borra - CPCPI 4900Contacto: Ary Moreno - MLS ID # 420701045-23VENTA COM

In [17]:
#Vemos la cantidad de nules en rooms antes de operar sobre la columna: 73830
df.isna().sum()

Unnamed: 0                         0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
dtype: int64

In [18]:
#Creo una nueva columna tomando el numero de ambientes según la descripcion:
df["strAmb"] = df.description.str.extract('([0-9]+)\s?amb',flags=re.IGNORECASE)

In [19]:
#Resultado:
df.loc[:,['strAmb','rooms','description','title']]

,strAmb,rooms,description,title
0,2,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PATIO GRANDE, CON LAVADERO, LIVING COMEDOR CON COCINA INTEGRADA, ARTEFACTO DE COCINA, ALACENA, BAÑO COMPLETO, Y DORMITORIO. TODO EN EXCELENTE ESTADO, PARA HABITAR.NO ES APTO CREDITO Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,NaN,NaN,"Venta de departamento en décimo piso al frente, a estrenar. Living comedor con ventanales hacia el balcón con pisos de madera. Cocina completa con doble mesada, muebles bajo mesada. Lavadero separado. Toilette.Dormitorio con placard. Segundo dormitorio a dividir. Calefacción por radiadorBaño completo.Balcón corrido. Posibilidad de cochera semicubierta. U$D 20.000 Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37
2,2,NaN,"2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON LAVADERO, BAÑO COMPLETO Y DORMITORIO CON PLACARD. BAJAS EXPENSAS, EXCELENTE UBICACION Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,3,NaN,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso.Reciclado a nuevo. ""Muy buen estado"" Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",PH 3 amb. cfte. reciclado
4,NaN,NaN,"DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURAL, AL CONTRAFRENTE, EDIFICIO DE POCAS UNIDADES Y MUY BIEN CONSERVADO. EN ESTADO ORIGINAL.-POSEE: 1 DORMITORIOS, 1 BAÑO COMPLETO, LIVING COMEDOR Y COCINA SEPARADA.-EN ESTADO ORIGINAL.- Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA MITRE
5,NaN,NaN,"Casa en el perímetro del barrio 338, ubicada en el extremo sur este del barrio, sobre calle 3 de caballería casi Nágera. Vista libre a terreno en frente.Consta de dos dormitorios, baño, living cocina comedor integrado de buen tamaño, lavadero con despensero en aluminio.Patio privado con churrasquera, tapiales y puerta de acceso de servicio. Frente con rejas y pequeño patio con verde, porch de acceso.Gas natural en la puerta. Aire acondicionado frío/calor. Baulera para guardado sobre pasillo. Muy buen estado en general de pisos, revestimientos, pintura, etc.Apto crédito. Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).","Casa Barrio 338. Sobre calle 3 de caballería, casi Nágera. Dos habit. Muy buen estado, patio con tapiales."
6,NaN,NaN,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENTE - SIN EXPENSAS! TODO EN PLANTA BAJA DOS DORMITORIOS (pisos de pinotea) - LIVING - COCINA COMEDOR - BAÑO - PATIO - GALPÓN Y LAVADERO EN EL PATIOEN MUY BUEN ESTADO - INMEJORABLE UBICACIÓN - A 1 CUADRA DE LA ESTACIÓN DE MUNRO - CERCA DE MAS DE 6 LINEAS DE COLECTIVOS - A 2 CUADRAS DEL CENTRO UNIVERSITARIO. 106 m2 TOTALES: 78 m2 CUBIERTOS Y 28 m2 DESCUBIERTOSAPTO PROFESIONAL Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO, EXCELENTE UBICACION EN MUNRO"
7,NaN,NaN,"EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER LUMINOSO ...PISO ALTO COCINA AMERICANA. BAÑO COMPLETO PILETA ,SOLARIUM, LAUNDRY APTO PROFESIONALPROXIMO AV DEL LIBERTADOR MUY BUENA ZONA !!!! Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).","JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CATEGORIA , VENTA"
8,NaN,NaN,"EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER LUMINOSO ...PISO ALTO COCINA AMERICANA. BAÑO COMPLETO PILETA ,SOLARIUM, LAUNDRY APTO PROFESIONALPROXIMO AV DEL LIBERTADOR MUY BUENA ZONA !!!! Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).","JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR , CATEGORIA , VENTA"
9,NaN,NaN,"MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWHITE 7637 SECTOR E MONOBLOK 67LISTO PARA ESCRI

In [20]:
#Hago lo mismo con el título:
df["strAmb2"] = df.title.str.extract('([0-9]+)\s?amb',flags=re.IGNORECASE)

In [21]:
#Vemos la cantidad de datos cargados en las nuevas columnas:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 non-null float64
price_usd_per_m2              68617 no

In [22]:
#Compruebo que titulo no da info adicional:
df[(df['strAmb']== np.nan) & (df['strAmb2']== np.nan)]
df[(df['strAmb']== np.nan) & (df['strAmb2']!= np.nan)]

,Unnamed: 0,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,strAmb,strAmb2


In [23]:
#Elimino la columna strAmb2
df.drop(columns=['strAmb2'], inplace=True)

In [24]:
#Creo un dict para transformar los números de letras a números:
busca_num = {'uno' : '1' , 'dos' : '2' , 'tres' : '3' , 'cuatro' : '4' ,
             'cinco' : '5' , 'seis' : '6' , 'siete' : '7' , 
             'mono' : '1' , 'mon' : '1'}

In [25]:
#Hago la transformación de letras a números:
df['strAmb'].replace(busca_num, inplace=True)

In [53]:
#Formateo la columna a float:
df['strAmb'] = df['strAmb'].astype(float)

In [61]:
#Antes de hacer la actualización masiva, hacemos una prueba en una columna nueva rooms2
df['rooms2'] = (np.where( df['strAmb'] < 11 , df['strAmb'], df['rooms']))

In [115]:
#Vemos que prevalece el dato a un NAN y strAmb antes que rooms:
df.loc[:,['rooms2','rooms','strAmb']]

,rooms2,rooms,strAmb
0,2.0,NaN,2.0
1,NaN,NaN,NaN
2,2.0,NaN,2.0
3,3.0,NaN,3.0
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [62]:
#Ahora hacemos la actualización en el campo rooms:
df['rooms'].fillna(value=df['rooms2'],inplace = True)

In [63]:
#Valido que se haya reemplazado en forma correcta
df.loc[:,['rooms2','rooms','strAmb']]

,rooms2,rooms,strAmb
0,2.0,2.0,2.0
1,NaN,NaN,NaN
2,2.0,2.0,2.0
3,3.0,3.0,3.0
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [64]:
#Elimino las columnas auxilares
df.drop(columns=['strAmb'], inplace=True)
df.drop(columns=['rooms2'], inplace=True)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 24 columns):
Unnamed: 0                    121220 non-null int64
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 non-null float64
price_usd_per_m2              68617 no